# Market Association Analysis

In [32]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [33]:
df_price=pd.read_csv("Price.csv")
df_price.head()

,ProductID,ProductName,Brand,Category_Level_0,Category_Level_1,Category_Level_2,Category_Level_3,Fiyat
0,1.0,Fairy Platinum Tablet,Fairy,Deterjan,Bulaşık Yıkama,Bulaşık Makinesi Ürünleri,Tablet Deterjanlar,60
1,2.0,Komili Riviera Yağı,Komili,"Gıda, Şekerleme",Sıvı Yağlar,Zeytinyağı,Riviera Zeytinyağı,40
2,3.0,Marmara Birlik Siyah Zeytin,Marmara Birlik,"Süt, Kahvaltılık",Zeytin,Siyah Zeytin,Özel Siyah Zeytin,20
3,4.0,CityFarm Organik 10'lu Yumurta,City Farm,"Süt, Kahvaltılık",Yumurta,Özel Yumurta,Özel Yumurta,20
4,5.0,Sütaş Pastörize Süt,Sütaş,"Süt, Kahvaltılık",Süt,Pastörize Süt,Pastörize Sade Süt,5


In [34]:
df_invoice=pd.read_csv("Invoice.csv")
df_invoice.head()

,Invoice ID,Invoice Date,Invoice_Total,Payment Method
0,1,23.03.2019,150,Debit
1,2,23.03.2019,65,Credit Card
2,3,23.03.2019,140,Debit
3,4,23.03.2019,75,Credit Card
4,5,23.03.2019,55,Credit Card


In [35]:
df_invoice_details=pd.read_csv("InvoiceDetails.csv")
df_invoice_details.head()

,InvoiceDetailID,InvoiceID,ProductID,Total,Adet
0,1,1,1,60,1.0
1,2,1,7,50,2.0
2,3,1,2,40,1.0
3,4,2,2,40,1.0
4,5,2,4,20,1.0


In [36]:
df=pd.merge(df_invoice_details, df_price, how='left', on=['ProductID']).\
drop(["Brand","Category_Level_0","Category_Level_1","Category_Level_2","Category_Level_3","Fiyat"],axis=1)
df.head(5)

,InvoiceDetailID,InvoiceID,ProductID,Total,Adet,ProductName
0,1,1,1,60,1.0,Fairy Platinum Tablet
1,2,1,7,50,2.0,Prima Premium Care İkiz Paket 5 No Junior 25'Li
2,3,1,2,40,1.0,Komili Riviera Yağı
3,4,2,2,40,1.0,Komili Riviera Yağı
4,5,2,4,20,1.0,CityFarm Organik 10'lu Yumurta


In [37]:
basket = (df.groupby(['InvoiceID', 'ProductName'])['Adet']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceID'))

In [38]:
basket.head()

ProductName,Beypazarı Maden Suyu 6X200 Ml,Bonne Maman Çilek Reçeli 370 G,CityFarm Organik 10'lu Yumurta,Coca-Cola Şekersiz 6X250ML Kutu,Doğuş Toz Şeker 1 Kg,Elma Fuji Kg,Fairy Platinum Tablet,Komili Riviera Yağı,Limon Nostalji File Kg,Marmara Birlik Siyah Zeytin,...,Nutella 750 G,Prima Premium Care İkiz Paket 5 No Junior 25'Li,Selpak Lavanta Tuvalet Kağıdı 32'li,Sütaş Pastörize Süt,Sütaş Tava Yoğurt 1000 G,Torku Tereyağ 500 G,Yayla Dermason Fasulye 1 Kg,Yayla Koçbaşı Nohut 1000 G,Yayla Kırmızı Mercimek 1 Kg,Çilek Kg
InvoiceID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [40]:
basket_sets.head()

ProductName,Beypazarı Maden Suyu 6X200 Ml,Bonne Maman Çilek Reçeli 370 G,CityFarm Organik 10'lu Yumurta,Coca-Cola Şekersiz 6X250ML Kutu,Doğuş Toz Şeker 1 Kg,Elma Fuji Kg,Fairy Platinum Tablet,Komili Riviera Yağı,Limon Nostalji File Kg,Marmara Birlik Siyah Zeytin,...,Nutella 750 G,Prima Premium Care İkiz Paket 5 No Junior 25'Li,Selpak Lavanta Tuvalet Kağıdı 32'li,Sütaş Pastörize Süt,Sütaş Tava Yoğurt 1000 G,Torku Tereyağ 500 G,Yayla Dermason Fasulye 1 Kg,Yayla Koçbaşı Nohut 1000 G,Yayla Kırmızı Mercimek 1 Kg,Çilek Kg
InvoiceID,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,1,1,0,1,...,1,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# set parameters for apriori algorithms 
### min_support = 0.1 (%10) 

In [41]:
frequent_itemsets = apriori(basket_sets, min_support=0.1, use_colnames=True)

In [42]:
frequent_itemsets

,support,itemsets
0,0.26,(Beypazarı Maden Suyu 6X200 Ml)
1,0.32,(CityFarm Organik 10'lu Yumurta)
2,0.10,(Doğuş Toz Şeker 1 Kg)
3,0.24,(Elma Fuji Kg)
4,0.30,(Fairy Platinum Tablet)
5,0.34,(Komili Riviera Yağı)
6,0.32,(Marmara Birlik Siyah Zeytin)
7,0.16,(Nutella 750 G)
8,0.20,(Prima Premium Care İkiz Paket 5 No Junior 25'Li)
9,0.24,(Selpak Lavanta Tuvalet Kağıdı 32'li)


# Support: Its the default popularity of an item. 
###### In mathematical terms, the support of item X is nothing but the ratio of transactions involving X to the total number of transactions.
###### Support(X) = (Transactions involving X)/(Total transaction)



# Confidence: Likelihood that customer who bought both X and Y. Its divides the number of transactions involving both X and Y by the number of transactions involving X.
######  Confidence(X => Y) = (Transactions involving both X and Y)/(Transactions involving only X)

# Lift Value:  Increase in the sale of A when you sell B.


###### Lift(A => B) = Confidence(A, B) / Support(B)
###### So, likelihood of a customer buying both A and B together is ‘lift-value’ times more than the chance if purchasing alone.
###### Lift (A => B) = 1 means that there is no correlation within the itemset.
###### Lift (A => B) > 1 means that there is a positive correlation within the itemset, i.e., products in the itemset, A, and B, are more likely to be bought together.
###### Lift (A => B) < 1 means that there is a negative correlation within the itemset, i.e., products in itemset, A, and B, are unlikely to be bought together.

## The confidence level for the rule is 0.3.

In [43]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
rules = rules.sort_values(['confidence'], ascending=False)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
19,(Nutella 750 G),(Komili Riviera Yağı),0.16,0.34,0.12,0.750000,2.205882,0.0656,2.640000
9,(Selpak Lavanta Tuvalet Kağıdı 32'li),(CityFarm Organik 10'lu Yumurta),0.24,0.32,0.14,0.583333,1.822917,0.0632,1.632000
20,(Selpak Lavanta Tuvalet Kağıdı 32'li),(Marmara Birlik Siyah Zeytin),0.24,0.32,0.14,0.583333,1.822917,0.0632,1.632000
13,(Fairy Platinum Tablet),(Marmara Birlik Siyah Zeytin),0.30,0.32,0.16,0.533333,1.666667,0.0640,1.457143
0,(Elma Fuji Kg),(Beypazarı Maden Suyu 6X200 Ml),0.24,0.26,0.12,0.500000,1.923077,0.0576,1.480000


# Results


## The support value for the first line is 0.12. This number is calculated by dividing the number of transactions containing ‘(Nutella 750 G)’  and ‘(Komili Riviera Yağı)’ by the total number of transactions.
## The confidence level for the first line is 0.75, which shows that out of all the transactions that contain  ‘Nutella 750 G’ , 75. percent contain ‘Komili Riviera Yağı’’ too.
## The lift of 2.205 tells us that ‘Komili Riviera Yağı’ is 2.205 times more likely to be bought by the customers who buy  ‘Nutella 750 G’ compared to the default likelihood sale of ‘Komili Riviera Yağı.’